# **Meta-Llama-3.1-8B-Instruct 모델 GPTQ 양자화하기**

# 1. 라이브러리 다운로드

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
!pip install -q -U peft accelerate optimum
!pip install datasets==2.15.0
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
!pip install huggingface_hub
!pip install langchain
!pip install transformers[torch] -U
!pip install torch --extra-index-url https://download.pytorch.org/whl/cu118


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.2.0
    Uninstalling fsspec-2024.2.0:
      Successfully uninstalled fsspec-2024.2.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstal

In [3]:
import torch

# CUDA 사용 가능 확인
print("Is CUDA available:", torch.cuda.is_available())

Is CUDA available: True


# 2. 지원되는 데이터셋을 전달하여 모델 양자화하기
- 'wikitest2' 데이터셋 사용
- 추후 재정정보 데이터셋 전처리 후 사용고려
- 4비트 정밀도로 양자화 (지원 : 2,4,6,8)

> **전처리 시 고려사항**
1. PyPDF2, pdfplumbler, pdfminer 같은 라이브러리로 pdf에서 텍스트 추출
2. 불필요한 공백, 페이지 번호, 제목 등 필요하지 않은 정보 제거
3. 순서가 맞지 않는 문장 재구성, 정제
4. 데이터셋 형태로 저장(JSON, CSV, TXT 등)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import logging
import torch

# 로깅 기본 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 모델 ID
model_id = "sh2orc/Llama-3.1-Korean-8B-Instruct" # 베이스 Llama 3 + 한국어 파인튜닝 모델

# GPTQ 양자화 설정
quantization_config = GPTQConfig(
    bits=4,
    group_size=128,       # 일반적으로 사용되는 그룹 크기
    dataset="wikitext2",  # 양자화에 사용할 기본 데이터셋, 추후 제공받은 데이터셋을 전처리하여 넣어주는 것도 고려하기
    desc_act=False
)

# 토크나이저 로드
logging.info(f"{model_id} 모델을 위한 토크나이저 로딩 중...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
logging.info("토크나이저 로딩 완료")

# 모델 로드 및 양자화
logging.info(f"{model_id} 모델 로딩 및 양자화 설정 적용 중: {quantization_config}")
quant_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",         # 자동으로 가능한 장치에 할당
    torch_dtype=torch.float16  # 메모리 절약을 위해 float16 사용
)
logging.info("모델 로딩 및 양자화 완료")

2024-08-13 12:13:29,834 - INFO - sh2orc/Llama-3.1-Korean-8B-Instruct 모델을 위한 토크나이저 로딩 중...
2024-08-13 12:13:30,320 - INFO - 토크나이저 로딩 완료
2024-08-13 12:13:30,321 - INFO - sh2orc/Llama-3.1-Korean-8B-Instruct 모델 로딩 및 양자화 설정 적용 중: GPTQConfig(quant_method=<QuantizationMethod.GPTQ: 'gptq'>)
2024-08-13 12:13:30,476 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Quantizing model.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

2024-08-13 12:14:33,346 - INFO - Start quantizing block model.layers 1/32
2024-08-13 12:14:33,347 - INFO - Module to quantize [['self_attn.q_proj'], ['self_attn.k_proj'], ['self_attn.v_proj'], ['self_attn.o_proj'], ['mlp.gate_proj'], ['mlp.up_proj'], ['mlp.down_proj']]


Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

2024-08-13 12:39:17,045 - INFO - Quantizing self_attn.q_proj in block 31/32...
2024-08-13 12:39:19,405 - INFO - duration: 2.3572022914886475
2024-08-13 12:39:19,408 - INFO - avg loss: 293.40142822265625
2024-08-13 12:39:21,555 - INFO - Quantizing self_attn.k_proj in block 31/32...
2024-08-13 12:39:23,878 - INFO - duration: 2.3209688663482666
2024-08-13 12:39:23,880 - INFO - avg loss: 194.94924926757812
2024-08-13 12:39:26,029 - INFO - Quantizing self_attn.v_proj in block 31/32...
2024-08-13 12:39:28,347 - INFO - duration: 2.316722869873047
2024-08-13 12:39:28,350 - INFO - avg loss: 103.46127319335938
2024-08-13 12:39:30,481 - INFO - Quantizing self_attn.o_proj in block 31/32...
2024-08-13 12:39:32,830 - INFO - duration: 2.346766948699951
2024-08-13 12:39:32,833 - INFO - avg loss: 13.251123428344727
2024-08-13 12:39:34,954 - INFO - Quantizing mlp.gate_proj in block 31/32...
2024-08-13 12:39:37,371 - INFO - duration: 2.414619207382202
2024-08-13 12:39:37,376 - INFO - avg loss: 764.360839

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

2024-08-13 12:40:06,520 - INFO - Quantizing self_attn.q_proj in block 32/32...
2024-08-13 12:40:08,871 - INFO - duration: 2.3486011028289795
2024-08-13 12:40:08,873 - INFO - avg loss: 232.31521606445312
2024-08-13 12:40:11,013 - INFO - Quantizing self_attn.k_proj in block 32/32...
2024-08-13 12:40:13,346 - INFO - duration: 2.3308284282684326
2024-08-13 12:40:13,349 - INFO - avg loss: 128.8953399658203
2024-08-13 12:40:15,496 - INFO - Quantizing self_attn.v_proj in block 32/32...
2024-08-13 12:40:17,825 - INFO - duration: 2.326690912246704
2024-08-13 12:40:17,827 - INFO - avg loss: 55.518653869628906
2024-08-13 12:40:19,955 - INFO - Quantizing self_attn.o_proj in block 32/32...
2024-08-13 12:40:22,332 - INFO - duration: 2.375446081161499
2024-08-13 12:40:22,334 - INFO - avg loss: 29.380741119384766
2024-08-13 12:40:24,459 - INFO - Quantizing mlp.gate_proj in block 32/32...
2024-08-13 12:40:26,841 - INFO - duration: 2.379107713699341
2024-08-13 12:40:26,843 - INFO - avg loss: 700.1906738

KeyboardInterrupt: 

선형 레이어의 속성을 확인하여 모델이 올바르게 정량화되었는지 확인할 수 있으며, 여기에는 torch.int32 dtype에 있어야 하는 qweight 및 qzeros 속성이 포함되어 있어야 합니다.

In [ ]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__

이제 잘 동작하는지 확인하기 위해 양자화된 모델에 대해 추론을 수행해 보겠습니다. (트랜스포머와 동일한 API를 사용할 것)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

text = "안녕"
inputs = tokenizer(text, return_tensors="pt").to(0)
out = quant_model.generate(**inputs)
print(tokenizer.decode(out[0], skip_special_tokens=True))
logging.info(f"생성된 텍스트: {result_text}")

# 3. 양자화된 모델 허깅페이스에 업로드하기

In [ ]:
from huggingface_hub import notebook_login

# Hugging Face Hub 로그인 (최초 1회 실행 필요)
notebook_login()

In [ ]:
# 양자화된 모델을 Hugging Face Hub에 업로드
quant_model.push_to_hub("ormor/Llama-3.1-Korean-8B-Instruct-GPTQ-4bit")
tokenizer.push_to_hub("ormor/Llama-3.1-Korean-8B-Instruct-GPTQ-4bit")

# 4. 허깅페이스에 업로드한 모델 가져오기

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline

def setup_llm_pipeline():
    # 모델 ID (Hugging Face Hub에서 가져온 양자화된 모델 ID)
    model_id = "ormor/Llama-3.1-Korean-8B-Instruct-GPTQ-4bit"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        trust_remote_code=True
    )
    
    print(f"#### [ model ] ####\n{model}\n###################")

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=450,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

파이프라인 설정 및 테스트

In [ ]:
# 파이프라인 설정
hf_pipeline = setup_llm_pipeline()

# 텍스트 생성 예시
text = "안녕"
result = hf_pipeline(text)
print(result)